In [1]:
# Load libraries
import pandas as pd
import numpy as np
!pip install plotly -U # Update Plotly for additonal tools -> Restart runtime after update
import plotly.express as px
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import seaborn as sns
import datetime
import glob
import os
from datetime import datetime
from functools import reduce
from google.colab import drive 
drive.mount('/content/gdrive')

Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (4.14.3)


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Finalised Dataset
Our finalised dataset comprises of 650 observations and 12 features.

In [2]:
# Pull dataset
rec_data = pd.read_csv("/content/gdrive/My Drive/EC4308/Project/Code & Data/Data/final_data.csv")
rec_data

,DATE,PAYEMS,FEDFUNDS,3MTB_SECONDARYMKT,GS1,GS5,GS10,CPI,DEBT_PUB,DEBT_HH,SP500,INDPRO,TCU,UNRATE,Is_Recession
0,1966-01-01,62529,4.42,4.59,4.88,4.86,4.61,0.000000,40.33999,0.000,0.000000,34.1729,0.0000,4.0,0
1,1966-02-01,62796,4.60,4.65,4.94,4.98,4.83,0.628931,40.33999,0.000,0.000000,34.3945,0.0000,3.8,0
2,1966-03-01,63192,4.66,4.59,4.97,4.92,4.87,0.312500,40.33999,0.000,0.000000,34.8652,0.0000,3.8,0
3,1966-04-01,63437,4.67,4.62,4.90,4.83,4.75,0.623053,39.26763,0.000,0.000000,34.9206,0.0000,3.8,0
4,1966-05-01,63712,4.90,4.64,4.93,4.89,4.78,0.000000,39.26763,0.000,0.000000,35.2529,0.0000,3.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,2019-10-01,151524,1.83,1.65,1.61,1.53,1.71,0.228619,106.68579,75.462,3037.560059,109.0270,76.9891,3.6,0
646,2019-11-01,151758,1.55,1.54,1.57,1.64,1.81,-0.053624,106.68579,75.462,3140.979980,110.0388,77.5723,3.6,0
647,2019-12-01,151919,1.55,1.54,1.55,1.68,1.86,-0.090977,106.68579,75.462,3230.780029,109.6527,77.1697,3.6,0
648,2020-01-01,152234,1.55,1.52,1.53,1.56,1.76,0.387977,107.71144,76.450,3225.520020,109.1845,76.8754,3.5,0


#### Get all recession periods
[Thx stack overflow](https://stackoverflow.com/questions/55696209/find-start-end-index-of-bouts-of-consecutive-equal-values) :) Cumulative sum trick.

In [3]:
v = (rec_data['Is_Recession'] != rec_data['Is_Recession'].shift()).cumsum()
u = rec_data.groupby(v)['Is_Recession'].agg(['all', 'count'])
m = u['all'] & u['count'].ge(3)

In [4]:
recession_periods_idx = rec_data.groupby(v).apply(lambda x: (x.index[0], x.index[-1]))[m]
recession_periods = [(rec_data.iloc[i, 0], rec_data.iloc[j, 0]) for i, j in recession_periods_idx]

## Handling Missing Data
No missing data found sans the CPI hitting 0 in certain instances and our limited historical data for S&P 500 and Household Debt.



In [5]:
rec_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DATE               650 non-null    object 
 1   PAYEMS             650 non-null    int64  
 2   FEDFUNDS           650 non-null    float64
 3   3MTB_SECONDARYMKT  650 non-null    float64
 4   GS1                650 non-null    float64
 5   GS5                650 non-null    float64
 6   GS10               650 non-null    float64
 7   CPI                650 non-null    float64
 8   DEBT_PUB           650 non-null    float64
 9   DEBT_HH            650 non-null    float64
 10  SP500              650 non-null    float64
 11  INDPRO             650 non-null    float64
 12  TCU                650 non-null    float64
 13  UNRATE             650 non-null    float64
 14  Is_Recession       650 non-null    int64  
dtypes: float64(12), int64(2), object(1)
memory usage: 76.3+ KB


## Feature Engineering
[TBD] Provide scientific/ economic/ mathematical rationale for features engineered.

### Interest rate spread: 10 year - 3 month (See Kauppi)
Detects yield curve inversions which precede major recessions such as the GFC of 2008. However, false alarms do happen as the yield curve inverted in 2019 but a recessions was not baked in until COVID-19 happened.

In [6]:
int_rate_spread = rec_data['GS10'] - rec_data['3MTB_SECONDARYMKT']
rec_data['10Y3MTH_SPREAD'] = int_rate_spread

In [7]:
fig = px.box(rec_data, x="Is_Recession", y="10Y3MTH_SPREAD", 
             color="Is_Recession",
             notched=True, # used notched shape
             title="Boxplot of Interest Rate Spread"
            )
fig.show()

Interest rate spread seems to dip more deeply into negative region during a recession. However, we also see a number of negative outliers with larger negative values when there is no recession (i.e. during the Expansion period). This might be due to a yield curve inversion occurring in the leading up to an actual recession.

In [8]:
fig = px.line(rec_data, x = 'DATE', y = '10Y3MTH_SPREAD')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

As observed, the spread always dips into negative territory preceding a recession. Even in 2019, we observe a yield curve inversion just before 2020 between the months of May till October, foreshadowing a possible recession in late 2019 to early 2020. Of which, was triggered due to COVID-19.

### Capturing momentum
#### Rolling Window (3 month)
We use a 3 month rolling window to capture condensed information on past values of a indicator variable.
##### 1) Non farm payrolls

In [9]:
rec_data['PAYEMS_ROLMEAN3'] = rec_data['PAYEMS'].rolling(window=3).mean()

In [10]:
fig = px.line(rec_data, x = 'DATE', y = 'PAYEMS_ROLMEAN3')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

Appears to lag the recession rather than lead the recession. May not be super useful.

##### 2) Fed Funds Rate

In [11]:
rec_data['FEDFUNDS_ROLMEAN3'] = rec_data['FEDFUNDS'].rolling(window=3).mean()

In [12]:
fig = px.line(rec_data, x = 'DATE', y = 'FEDFUNDS_ROLMEAN3')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

Appears to hit a peak right before the onset of a recession.

##### 3) 10 year - 3 month interest rate spread

In [13]:
rec_data['10Y3MTH_SPREAD_ROLMEAN3'] = rec_data['10Y3MTH_SPREAD'].rolling(window=3).mean()

In [14]:
fig = px.line(rec_data, x = 'DATE', y = '10Y3MTH_SPREAD_ROLMEAN3')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

Like its unadultered interest rate spread indicator, it dips below the 0 mark prior to a recession but the values are averaged out and thus less pronounced. How would a less pronounced yield curve affect predictive accuracy.

##### 4) CPI

In [15]:
rec_data['CPI_ROLMEAN3'] = rec_data['CPI'].rolling(window=3).mean()

In [16]:
fig = px.line(rec_data, x = 'DATE', y = 'CPI_ROLMEAN3')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

Appears to consistently be on the rise before a recession except for the 1981 recession.

##### 5) Public Debt

In [17]:
rec_data['DEBT_PUB_ROLMEAN3'] = rec_data['DEBT_PUB'].rolling(window=3).mean()

In [18]:
fig = px.line(rec_data, x = 'DATE', y = 'DEBT_PUB_ROLMEAN3')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

General rising trend, but could be due to exogenous factors apart from economic climate. Appears to and naturally rises during a recession, but no clear and consistent behvaiour prior to a recession.

##### 6) Household Debt

In [19]:
rec_data['DEBT_HH_ROLMEAN3'] = rec_data['DEBT_HH'].rolling(window=3).mean()

In [20]:
fig = px.line(rec_data, x = 'DATE', y = 'DEBT_HH_ROLMEAN3')
fig.add_hline(y = 0.0)
# Add in recession periods
# Only tracked prior to the GFC circa 2007-8
start, end = recession_periods[-1]
fig.add_vrect(x0=start, x1=end, 
              annotation_text="recession", annotation_position="top left",
              fillcolor="red", opacity=0.25, line_width=1)
fig.show()

Too small a sample size to determine utility, but naturally rose due to households accruing bad debt (subprime mortgage crisis).

##### 7) S&P 500 Index

In [21]:
rec_data['SP500_ROLMEAN3'] = rec_data['SP500'].rolling(window=3).mean()

In [22]:
fig = px.line(rec_data, x = 'DATE', y = 'SP500_ROLMEAN3')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

Observably hits a peak right before a recession. However, as observed the during COVID-19 recession, the stock market was floated up by 'over-valued' tech stocks.

##### 8) Industrial Production

In [23]:
rec_data['INDPRO_ROLMEAN3'] = rec_data['INDPRO'].rolling(window=3).mean()

In [24]:
fig = px.line(rec_data, x = 'DATE', y = 'INDPRO_ROLMEAN3')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

Occasionally precedes the recession (E.g. the GFC and early 2000s recession). In other cases, the peak seems to come early into the recession period.

##### 9) Capacity Utilisation

In [25]:
rec_data['TCU_ROLMEAN3'] = rec_data['TCU'].rolling(window=3).mean()

In [26]:
fig = px.line(rec_data, x = 'DATE', y = 'TCU_ROLMEAN3')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

Occasionally precedes the recession (E.g. the GFC and early 2000s recession). In other cases, the peak seems to come early into the recession period.

##### 10) Unemployment Rate

In [27]:
rec_data['UNRATE_ROLMEAN3'] = rec_data['UNRATE'].rolling(window=3).mean()

In [28]:
fig = px.line(rec_data, x = 'DATE', y = 'UNRATE_ROLMEAN3')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

Unemployment trough comes before the recession, however, there may be multiple false positives, as it could hit a local minimum and recover before dropping again. However, it seems reliable.

#### Percentage Change (3 month)
Nothing more than 3 months so we do not lose too many observations.
##### 1) Non farm payrolls

In [29]:
rec_data['PAYEMS_3MTHCHANGE'] = rec_data['PAYEMS'].pct_change(periods = 3)
fig = px.line(rec_data, x = 'DATE', y = 'PAYEMS_3MTHCHANGE')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

##### 2) Fed Funds Rate

In [30]:
rec_data['FEDFUNDS_3MTHCHANGE'] = rec_data['FEDFUNDS'].pct_change(periods = 3)
fig = px.line(rec_data, x = 'DATE', y = 'FEDFUNDS_3MTHCHANGE')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

##### 3) Interest rate spread

In [31]:
rec_data['10Y3MTH_SPREAD_3MTHCHANGE'] = rec_data['10Y3MTH_SPREAD'].pct_change(periods = 3)
fig = px.line(rec_data, x = 'DATE', y = '10Y3MTH_SPREAD_3MTHCHANGE')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

##### 4) Inflation

In [32]:
rec_data['CPI_3MTHCHANGE'] = rec_data['CPI'].pct_change(periods = 3)
fig = px.line(rec_data, x = 'DATE', y = 'CPI_3MTHCHANGE')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

##### 5) Public debt

In [33]:
rec_data['DEBT_PUB_3MTHCHANGE'] = rec_data['DEBT_PUB'].pct_change(periods = 3)
fig = px.line(rec_data, x = 'DATE', y = 'DEBT_PUB_3MTHCHANGE')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

##### 6) Household Debt

In [34]:
rec_data['DEBT_HH_3MTHCHANGE'] = rec_data['DEBT_HH'].pct_change(periods = 3)
fig = px.line(rec_data, x = 'DATE', y = 'DEBT_HH_3MTHCHANGE')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

##### 7) S&P 500

In [35]:
rec_data['SP500_3MTHCHANGE'] = rec_data['SP500'].pct_change(periods = 3)
fig = px.line(rec_data, x = 'DATE', y = 'SP500_3MTHCHANGE')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

##### 8) Industrial Production

In [36]:
rec_data['INDPRO_3MTHCHANGE'] = rec_data['INDPRO'].pct_change(periods = 3)

In [37]:
fig = px.line(rec_data, x = 'DATE', y = 'INDPRO_3MTHCHANGE')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

Occasionally dips into negative region preceding a recession and bottoms out during a recession.

##### 9) Capacity Utilisation

In [38]:
rec_data['TCU_3MTHCHANGE'] = rec_data['TCU'].pct_change(periods = 3)

In [39]:
fig = px.line(rec_data, x = 'DATE', y = 'TCU_3MTHCHANGE')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

Consistently dips into negative territory preceding a recession

##### 10) Unemployment Rate

In [40]:
rec_data['UNRATE_3MTHCHANGE'] = rec_data['UNRATE'].pct_change(periods = 3)

In [41]:
fig = px.line(rec_data, x = 'DATE', y = 'UNRATE_3MTHCHANGE')
fig.add_hline(y = 0.0)
# Add in recession periods
for start, end in recession_periods:
    fig.add_vrect(x0=start, x1=end, 
                annotation_text="recession", annotation_position="top left",
                fillcolor="red", opacity=0.25, line_width=1)
fig.show()

Erratic but trends upwards right before a recession, but may be too noisy a variable to add sufficient predictive power.

#### Removing less promising variables


*   PAYEMS_ROLMEAN3
*   10Y3MTH_SPREAD_ROLMEAN3
*   CPI_ROLMEAN3
*   DEBT_PUB_ROLMEAN3
*   DEBT_HH_ROLMEAN3
*   SP500_ROLMEAN3
*   TCU_ROLMEAN3
*   FEDFUNDS_3MTHCHANGE
*   CPI_3MTHCHANGE
*   DEBT_PUB_3MTHCHANGE
*   DEBT_HH_3MTHCHANGE
*   UNRATE_3MTHCHANGE
*   SP500_3MTHCHANGE



In [42]:
rec_data_final = rec_data.drop(columns=['PAYEMS_ROLMEAN3',
                                        '10Y3MTH_SPREAD_ROLMEAN3',
                                        'CPI_ROLMEAN3',
                                        'DEBT_PUB_ROLMEAN3',
                                        'DEBT_HH_ROLMEAN3',
                                        'SP500_ROLMEAN3',
                                        'TCU_ROLMEAN3',
                                        'FEDFUNDS_3MTHCHANGE',
                                        'CPI_3MTHCHANGE',
                                        'DEBT_PUB_3MTHCHANGE',
                                        'DEBT_HH_3MTHCHANGE',
                                        'UNRATE_3MTHCHANGE',
                                        'SP500_3MTHCHANGE'])

In [43]:
rec_data_final.describe()

,PAYEMS,FEDFUNDS,3MTB_SECONDARYMKT,GS1,GS5,GS10,CPI,DEBT_PUB,DEBT_HH,SP500,INDPRO,TCU,UNRATE,Is_Recession,10Y3MTH_SPREAD,FEDFUNDS_ROLMEAN3,INDPRO_ROLMEAN3,UNRATE_ROLMEAN3,PAYEMS_3MTHCHANGE,10Y3MTH_SPREAD_3MTHCHANGE,INDPRO_3MTHCHANGE,TCU_3MTHCHANGE
count,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,648.000000,648.000000,648.000000,647.000000,647.000000,647.000000,638.000000
mean,109952.096923,5.192938,4.683585,5.199431,5.904800,6.254831,0.323650,57.973210,24.246409,737.400707,72.386565,78.727433,6.007692,0.140000,1.571246,5.199630,72.388521,6.014763,0.004125,0.258340,0.005494,inf
std,25866.862871,3.798318,3.281827,3.493998,3.194377,2.956403,0.361249,23.848271,39.172512,799.727120,24.328727,11.570504,1.674720,0.347254,1.251619,3.781355,24.274057,1.668118,0.005326,5.929470,0.015850,NaN
min,62529.000000,0.070000,0.010000,0.100000,0.620000,1.500000,-1.915290,30.603330,0.000000,0.000000,34.172900,0.000000,3.400000,0.000000,-2.650000,0.073333,34.477533,3.400000,-0.017258,-19.846154,-0.079827,-0.084595
25%,89063.750000,1.985000,1.862500,2.240000,3.292500,4.132500,0.122737,35.748220,0.000000,0.000000,50.349300,77.132425,4.800000,0.000000,0.572500,1.999167,50.484425,4.800000,0.002429,-0.272839,-0.000075,-0.006773
50%,109925.000000,5.240000,4.925000,5.400000,5.930000,6.105000,0.295794,56.241190,0.000000,439.485001,66.665050,80.225800,5.700000,0.000000,1.635000,5.245000,66.648400,5.700000,0.004557,-0.046296,0.006783,0.000900
75%,132356.750000,7.217500,6.410000,7.132500,7.757500,7.867500,0.520833,64.385840,76.064000,1257.154968,96.029025,83.362150,7.200000,0.000000,2.570000,7.287500,95.847675,7.200000,0.007002,0.186605,0.014037,0.007812
max,152523.000000,19.100000,16.300000,16.720000,15.930000,15.320000,1.805869,107.711440,99.821000,3230.780029,110.551600,89.390200,10.800000,1.000000,4.420000,18.886667,110.392267,10.666667,0.019372,126.000000,0.046804,inf


In [44]:
rec_data_final.DATE

0      1966-01-01
1      1966-02-01
2      1966-03-01
3      1966-04-01
4      1966-05-01
          ...    
645    2019-10-01
646    2019-11-01
647    2019-12-01
648    2020-01-01
649    2020-02-01
Name: DATE, Length: 650, dtype: object

### Get Partial Autocorrelation of Features to determine 'rough' lagged value to include (To be fine tuned using AIC/ BIC)

## EDA
1. Boxplots to show distribution of values
2. Histograms to show differencein distribution between expansion vs recession
3. Autocorrelation across lags
4. Stationarity? (Do our models depend on stationarity?)
5. Pairplot of correlations and breakdown by Recession/ Expansion

## Class Imbalance

In [45]:
px.histogram(rec_data, x = 'Is_Recession',
             title='Class frequencies (Expansions Vs. Recessions)',
             color='Is_Recession',
             opacity=0.8)

### Overall Distribution of Values across all Features (Boxplots grouped by Recession Indicator)